In [ ]:
#pwd
cd Data

In [ ]:
from census import Census
from us import states
import pandas as pd

## Simple Call ##
'''Example US Census API Call
* The following pulls in Total Population('B05001_001E') for California for 2022(years can be specified) into a Dataframe.
* This should run relatively quick and efficient. Approx 2minutes or less.
'''
# Your Census API key
API_KEY = "Enter Key" #Change as necessary

# Initialize the Census API
c = Census(API_KEY)

# Query ACS 5-Year Data for 2022
data = c.acs5.state(
    ('B05001_001E'),  #Total population
    '06')

# Convert to pandas DataFrame
df = pd.DataFrame(data)

# Save the results to a CSV file
output_file = "acs5_ca_data.csv"
df.to_csv(output_file, index=False)

# Explore the data
print(df)

In [ ]:

# Load dependencies
import pandas as pd
from census import Census
from us import states

## A more detailed Call ##
'''Example US Census by county API Call
* The following pulls in Total Population('B05001_001E') for Washington counties for 2015,2022(years can be specified) into a Dataframe.
* This pulls in all county level data for Washington for the specified years. For variables NAME and 'B05001_001E'. NAME is simply the full county name plus subdivision name.
* 
'''
# Your Census API key
key = "Enter Key" #Change this as necessary.
c = Census(key)
# Choose relevant census dataset
dataset = c.acs5
# Assign variable codes and their corresponding labels
variables = ('NAME', 'B05001_001E')
labels =    ('NAME', 'Place of Birth for the Foreign-Born Population in the United States')
# Choose desired geographic aggregation level
geo = 'county subdivision:*'

# Year
year = 2015 #2015, 2022

# Choose filter criteria (geographic extent)
county_code = '033'
criteria = f'state:{states.WA.fips} county:{county_code}'
# Run query and store as DataFrame
r = dataset.get(variables,
          { 'for': geo,
            'in' : criteria},
               year = year)
df = pd.DataFrame(r).rename(columns={v: l for v, l in zip(variables, labels)})
print(df.head())

In [ ]:
import pandas as pd
from census import Census
from us import states
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import logging

'''
* The following code pull in Census Data using the py API. Using the us library just like with previous calls, and the census library. I pulled in all avaiable data for all the years in ACS5 for all available counties.
* All of the below variables were used. 
* It is important to note that these variables were seletect based on interest and some randonmly.
* There is inconsitencies between some variables existing in some years and not in others. So in the final output, you will potentially notice some missing years.
* The API itself was somewhat glitchy or unreliable at times. In come instances It stopped on for inputting to many variables. This needs to be explored further.
'''

# Configure logging to write errors to a file
logging.basicConfig(
    filename="errors.log",
    level=logging.ERROR,
    format="%(asctime)s - %(message)s"
)

# Load in your API key; replace API_KEY with your actual token
key = "Enter Key"  # Replace with your Census API key
c = Census(key)

# Choose relevant census dataset
dataset = c.acs5

# Assign variable codes
'''variables = (
    "B05001_001E", "B05001_002E", "B05001_003E", "B05002_001E", "B05002_002E",
    "B05002_003E", "B05002_004E", "B05002_005E", "B05002_006E", "B05003_001E",
    "B05003_002E", "B05003_003E", "B05003_004E", "B05005_001E", "B05005_002E",
    "B05005_003E", "B05006_001E", "B05006_002E", "B05007_001E", "B05007_002E",
    "B05007_003E", "B05010_001E", "B06007_001E", "B06007_004E", "B06007_005E",
    "B06007_006E", "B06008_001E", "B06008_002E", "B06009_001E", "B06009_005E",
    "B16005_001E", "B16010_001E", "B17021_001E", "B17021_002E", "B17021_003E",
    "B22001_002E", "B22001_003E", "B01001_001E", "B01002_001E", "B01002_002E",
    "B01002_003E", "B03002_001E", "B03002_003E", "B03002_004E", "B03002_006E",
    "B03002_012E", "B15003_022E", "B15003_023E", "B15003_001E", "B17020_001E",
    "B15003_024E", "B15003_025E", "B17020_002E", "B17020_003E", "B15003_017E",
    "B19013_001E", "B19001_001E", "B19001_002E", "B19001_017E", "B19301_001E",
    "B20005_001E", "B20005_002E", "B20005_003E", "B20005_004E", "B20005_005E",
    "B20005_006E", "B25001_001E", "B02001_001E", "B02001_002E", "B02001_003E",
    "B02001_004E", "B08006_001E", "B08006_002E", "B08006_003E", "B08006_008E",
    "B11001_001E", "B11001_002E", "B11001_003E", "B13002_001E", "B13002_002E",
    "B23001_001E", "B23001_002E", "B23001_007E", "B23001_008E", "B23001_009E",
    "B25004_001E", "B25004_002E", "B25004_003E", "B25004_004E", "B25004_005E",
    "B25004_006E", "B25004_007E", "B23025_001E", "B23025_002E", "B23025_003E",
    "B23025_004E", "B23025_005E", "B25035_001E", "B25077_001E", "B25064_001E",
    "B25105_001E", "B08014_001E", "B08014_002E", "B08014_003E", "B08014_004E",
    "B08014_005E", "B08014_006E", "B08014_007E", "B08014_008E", "B08014_009E",
    "B08014_010E", "B08014_011E", "B25032_001E", "B25032_002E", "B25032_003E",
    "B17017_001E", "B14002_001E", "B14002_002E", "B14002_003E", "B14002_004E",
    "B14002_005E", "B14002_006E", "B01001_002E", "B01001_026E", "B09001_001E",
    "B09002_001E", "B16001_001E", "B16001_002E", "B16001_003E", "B16001_004E",
    "B17001_002E", "B17001_031E", "B19313_001E", "B23006_001E", "B23006_002E",
    "B23006_003E", "B23006_004E", "B23006_005E", "B25002_001E", "B25002_002E",
    "B25063_001E", "B25071_001E", "B25087_001E", "B25091_001E", "B27001_001E",
    "B27001_002E", "B27001_003E", "B08121_001E", "B08121_002E", "B08121_003E",
    "B08121_004E", "B05006_003E", "B05006_005E", "B05006_008E", "B05007_004E",
    "B05007_005E", "B05001_005E", "B05001_006E", "B17025_002E", "B17025_004E",
    "B23006_007E", "B15012_002E", "B15012_003E", "B25075_002E", "B27020_003E",
    "B05001_001M","B05001_002M","B05001_003M","B05002_001M","B05002_002M",
    "B05002_003M","B05002_004M","B05002_005M","B05002_006M","B05003_001M",
    "B05003_002M","B05003_003M","B05003_004M","B05005_001M","B05005_002M",
    "B05005_003M","B05006_001M","B05006_002M","B05007_001M","B05007_002M",
    "B05007_003M","B05010_001M","B06007_001M","B06007_004M","B06007_005M",
    "B06007_006M","B06008_001M","B06008_002M","B06009_001M","B06009_005M",
    "B16005_001M","B16010_001M","B17021_001M","B17021_002M","B17021_003M",
    "B22001_002M","B22001_003M","B01001_001M","B01002_001M","B01002_002M",
    "B01002_003M","B03002_001M","B03002_003M","B03002_004M","B03002_006M",
    "B03002_012M","B15003_022M","B15003_023M","B15003_001M","B17020_001M",
    "B15003_024M","B15003_025M","B17020_002M","B17020_003M","B15003_017M",
    "B19013_001M","B19001_001M","B19001_002M","B19001_017M","B19301_001M",
    "B20005_001M","B20005_002M","B20005_003M","B20005_004M","B20005_005M",
    "B20005_006M","B25001_001M","B02001_001M","B02001_002M","B02001_003M",
    "B02001_004M","B08006_001M","B08006_002M","B08006_003M","B08006_008M",
    "B11001_001M","B11001_002M","B11001_003M","B13002_001M","B13002_002M",
    "B23001_001M","B23001_002M","B23001_007M","B23001_008M","B23001_009M",
    "B25004_001M","B25004_002M","B25004_003M","B25004_004M","B25004_005M",
    "B25004_006M","B25004_007M","B23025_001M","B23025_002M","B23025_003M",
    "B23025_004M","B23025_005M","B25035_001M","B25077_001M","B25064_001M",
    "B25105_001M","B08014_001M","B08014_002M","B08014_003M","B08014_004M",
    "B08014_005M","B08014_006M","B08014_007M","B08014_008M","B08014_009M",
    "B08014_010M","B08014_011M","B25032_001M","B25032_002M","B25032_003M",
    "B17017_001M","B14002_001M","B14002_002M","B14002_003M","B14002_004M",
    "B14002_005M","B14002_006M","B01001_002M","B01001_026M","B09001_001M",
    "B09002_001M","B16001_001M","B16001_002M","B16001_003M","B16001_004M",
    "B17001_002M","B17001_031M","B19313_001M","B23006_001M","B23006_002M",
    "B23006_003M","B23006_004M","B23006_005M","B25002_001M","B25002_002M",
    "B25063_001M","B25071_001M","B25087_001M","B25091_001M","B27001_001M",
    "B27001_002M","B27001_003M","B08121_001M","B08121_002M","B08121_003M",
    "B08121_004M","B05006_003M","B05006_005M","B05006_008M","B05007_004M",
    "B05007_005M","B05001_005M","B05001_006M","B17025_002M","B17025_004M",
    "B23006_007M","B15012_002M","B15012_003M","B25075_002M","B27020_003M"
)'''
# New Variables, some are the duplicates. So beware.
variables = ["B05001_001E", "B05001_002E", "B05001_003E", "B05001_004E", "B05001_005E", "B05001_006E",
    "B05002_001E", "B05002_002E", "B05002_003E", "B05002_004E", "B05002_005E", "B05002_006E",
    "B05002_007E", "B05002_008E", "B05002_009E", "B05002_010E", "B05002_011E", "B05002_012E",
    "B05002_013E", "B05002_014E", "B05002_015E", "B05003_001E", "B05003_002E", "B05003_003E",
    "B05003_004E", "B05003_005E", "B05003_006E", "B05003_007E", "B05003_008E", "B05003_009E",
    "B05003_010E", "B05003_011E", "B05003_012E", "B05003_013E", "B05003_014E", "B05003_015E",
    "B05003_016E", "B05003_017E", "B05003_018E", "B05003_019E", "B05003_020E", "B05003_021E",
    "B05003_022E", "B05003_023E", "B05003A_001E", "B05003A_023E", "B05003B_001E", "B05003B_023E",
    "B05003C_001E", "B05003C_023E", "B05003D_001E",
    "B05003D_023E", "B05003E_001E", "B05003E_023E", "B05003F_001E", "B05003F_023E",
    "B05003H_001E", "B05003H_023E", "B05003I_001E", "B05003I_023E"
]
'''
* The API call for some reason was breaking when using more than 150+ variable codes. It was very unstable, so dcreased the number to around 150. The above variable codes worked for me.
* Excluded after testing some manually: B11012_001E, B11012_002E, B11012_003E, B11012_004E, B11012_005E, B24010_002E, B16002_002E, B16002_003E, B16002_004E
* E: Estimate
* M: Margin of Error
'''

# Initialize an empty list to collect results
all_data = []

# Available ACS 5-Year dataset years
years = list(range(2010, 2023))  # Adjust year range as needed
#years = [2015, 2016]

# Function to process data for a single state in a year
def fetch_state_data(state, year):
    try:
        state_fips = state.fips
        state_name = state.name

        # Fetch county list for the current state
        counties = dataset.get(('NAME',), {'for': 'county:*', 'in': f'state:{state_fips}'}, year=year)
        state_data = []

        # Placeholder with all variables initialized to NA
        placeholder = {var: "NA" for var in variables}
        placeholder.update({
            "year": year,
            "state_fips": state_fips,
            "state_name": state_name,
            "county_fips": None,
            "county_name": None
        })

        # Loop through counties in the state
        for county in counties:
            county_code = county['county']
            county_name = county['NAME']
            try:
                r = dataset.get(
                    variables,
                    {'for': 'county subdivision:*', 'in': f'state:{state_fips} county:{county_code}'},
                    year=year
                )
                for row in r:
                    row.update({
                        "year": year,
                        "state_fips": state_fips,
                        "state_name": state_name,
                        "county_fips": county_code,
                        "county_name": county_name
                    })
                    # Fill missing variables with NA
                    complete_row = {**placeholder, **row}
                    state_data.append(complete_row)
            except Exception as e:
                logging.error(f"Error fetching data for county {county_name} ({county_code}) in state {state_name} for year {year}: {e}")
        return state_data
    except Exception as e:
        logging.error(f"Error fetching counties for state {state.name} in year {year}: {e}")
        return []

# Parallel processing for years
for year in tqdm(years, desc="Processing Years"):
    try:
        with ThreadPoolExecutor(max_workers=15) as executor:
            # Submit state processing tasks in parallel
            futures = [executor.submit(fetch_state_data, state, year) for state in states.STATES]

            # Collect results as tasks complete
            for future in as_completed(futures):
                all_data.extend(future.result())
    except Exception as e:
        logging.error(f"Error processing year {year}: {e}")

# Convert collected data to a DataFrame
df = pd.DataFrame(all_data)

# Reorder columns to place state and county information first
column_order = ['year', 'state_fips', 'state_name', 'county_fips', 'county_name'] + [col for col in df.columns if col not in ['year', 'state_fips', 'state_name', 'county_fips', 'county_name']]
df = df[column_order]

# Save the results to a CSV file
output_file = "acs5_immigration_foreign_allyears.csv"
df.to_csv(output_file, index=False)

#parquet_path = "acs5_immigration_foreign_allyears.parquet"
#df.to_parquet(parquet_path, index=False)

# Confirm completion
print(f"Data collection complete. Results saved to {output_file}")
print(df.head())

In [ ]:
#######################################################################
#######################################################################
#######################################################################

In [ ]:
# IMPORTANT: I used this script for testing variable codes and the script. Code is the same as previous(above), except for the halt execution to stop when code runs into nonexistent variable codes for years.
import pandas as pd
from census import Census
from us import states
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import logging
import sys

# Configure logging to write errors to a file
logging.basicConfig(
    filename="errors.log",
    level=logging.ERROR,
    format="%(asctime)s - %(message)s"
)

# Load in your API key; replace API_KEY with your actual token
key = "Enter Key"  # Replace with your Census API key
c = Census(key)

# Choose relevant census dataset
dataset = c.acs5

# Assign variable codes
'''variables = ("B27001_002E", "B27001_003E", "B08121_001E", "B08121_002E", "B08121_003E",
             "B08121_004E", "B05006_003E", "B05006_005E", "B05006_008E", "B05007_004E",
             "B05007_005E", "B05001_005E", "B05001_006E", "B17025_002E", "B17025_004E",
             "B23006_007E", "B15012_002E", "B15012_003E", "B25075_002E", "B27020_003E",
             "B07003_001E", "B07003_003E"
            )'''
variables = [
    "B05001_001E", "B05001_002E", "B05001_003E", "B05001_004E", "B05001_005E", "B05001_006E",
    "B05002_001E", "B05002_002E", "B05002_003E", "B05002_004E", "B05002_005E", "B05002_006E",
    "B05002_007E", "B05002_008E", "B05002_009E", "B05002_010E", "B05002_011E", "B05002_012E",
    "B05002_013E", "B05002_014E", "B05002_015E", "B05003_001E", "B05003_002E", "B05003_003E",
    "B05003_004E", "B05003_005E", "B05003_006E", "B05003_007E", "B05003_008E", "B05003_009E",
    "B05003_010E", "B05003_011E", "B05003_012E", "B05003_013E", "B05003_014E", "B05003_015E",
    "B05003_016E", "B05003_017E", "B05003_018E", "B05003_019E", "B05003_020E", "B05003_021E",
    "B05003_022E", "B05003_023E", "B05003A_001E", "B05003A_023E", "B05003B_001E", "B05003B_023E",
    "B05003C_001E", "B05003C_023E", "B05003D_001E",
    "B05003D_023E", "B05003E_001E", "B05003E_023E", "B05003F_001E", "B05003F_023E",
    "B05003H_001E", "B05003H_023E", "B05003I_001E", "B05003I_023E"
]
#) #The AAPI call for some reason was breaking when using more than 150+ variable codes. It was very unstable, so dcreased the number to around 150. The above variable codes worked for me.
# Excluded after testing some manually: B11012_001E, B11012_002E, B11012_003E, B11012_004E, B11012_005E, B24010_002E, B16002_002E, B16002_003E, B16002_004E

# Available ACS 5-Year dataset years
years = list(range(2010, 2023))  # Adjust year range as needed
#years = [2015, 2022]  # Adjust as needed

# Initialize an empty list to collect results
all_data = []

# Function to process data for a single state in a year
def fetch_state_data(state, year):
    try:
        state_fips = state.fips
        state_name = state.name

        # Fetch county list for the current state
        counties = dataset.get(('NAME',), {'for': 'county:*', 'in': f'state:{state_fips}'}, year=year)
        state_data = []

        # Placeholder with all variables initialized to NA
        placeholder = {var: "NA" for var in variables}
        placeholder.update({
            "year": year,
            "state_fips": state_fips,
            "state_name": state_name,
            "county_fips": None,
            "county_name": None
        })

        # Loop through counties in the state
        for county in counties:
            county_code = county['county']
            county_name = county['NAME']
            try:
                r = dataset.get(
                    variables,
                    {'for': 'county subdivision:*', 'in': f'state:{state_fips} county:{county_code}'},
                    year=year
                )
                if not r:  # If API response is empty
                    logging.warning(f"No data returned for county {county_name} ({county_code}) in state {state_name} for year {year}.")
                    placeholder.update({
                        "county_fips": county_code,
                        "county_name": county_name
                    })
                    state_data.append(placeholder.copy())
                else:
                    for row in r:
                        row.update({
                            "year": year,
                            "state_fips": state_fips,
                            "state_name": state_name,
                            "county_fips": county_code,
                            "county_name": county_name
                        })
                        # Fill missing variables with NA
                        complete_row = {**placeholder, **row}
                        state_data.append(complete_row)
            except Exception as e:
                error_message = f"Nonexistent variable encountered for county {county_name} ({county_code}) in state {state_name} for year {year}: {e}"
                logging.error(error_message)
                sys.exit(error_message)
        return state_data
    except Exception as e:
        logging.error(f"Error fetching counties for state {state.name} in year {year}: {e}")
        sys.exit(f"Error fetching counties for state {state.name} in year {year}: {e}")

# Parallel processing for years
for year in tqdm(years, desc="Processing Years"):
    try:
        with ThreadPoolExecutor(max_workers=10) as executor:
            # Submit state processing tasks in parallel
            futures = [executor.submit(fetch_state_data, state, year) for state in states.STATES]

            # Collect results as tasks complete
            for future in as_completed(futures):
                all_data.extend(future.result())
    except Exception as e:
        logging.error(f"Error processing year {year}: {e}")
        sys.exit(f"Error processing year {year}: {e}")

# Convert collected data to a DataFrame
df = pd.DataFrame(all_data)

# Reorder columns to place state and county information first
default_columns = ['year', 'state_fips', 'state_name', 'county_fips', 'county_name'] + list(variables)
for col in default_columns:
    if col not in df.columns:
        df[col] = "NA"

column_order = ['year', 'state_fips', 'state_name', 'county_fips', 'county_name'] + [col for col in df.columns if col not in ['year', 'state_fips', 'state_name', 'county_fips', 'county_name']]
df = df[column_order]

# Save the results to a CSV file
output_file = "acs5_immigration_foreign_allyears.csv"
df.to_csv(output_file, index=False)

#parquet_path = "acs5_immigration_foreign_allyears.parquet"
#df.to_parquet(parquet_path, index=False)

# Confirm completion
print(f"Data collection complete. Results saved to {output_file}")
print(df.head())

In [ ]:
# Merging two Census dataframes. After identfying some inconsistencies with the first pulled variable codes. 
import pandas as pd

# Load the CSV files into DataFrames
file1_path = 'acs5_immigration_foreign_allyears_new.csv'  # Replace with the actual path to your first CSV file
file2_path = 'acs5_immigration_foreign_allyears.csv'  # Replace with the actual path to your second CSV file

# Read the CSV files
df1 = pd.read_csv(file1_path)
df2 = pd.read_csv(file2_path)

# Merge the DataFrames on 'year' and 'county_name'
merged_df = pd.merge(df1, df2, on=['year','state_fips','state_name','county_fips','county_name', 'GEO_ID', 'state', 'county', 'county subdivision'], how='inner')

# Output the merged DataFrame to a new CSV file
output_path = 'acs5_immigration_foreign_allyears_final.csv'  # Replace with your desired output path
merged_df.to_csv(output_path, index=False)

print(f"Merged file saved to: {output_path}")
print(merged_df.head())
